# 1.TRPO代理目标函数的解释
## 1.1 数学符号的介绍
### 1.1.1 数学期望 $\mathbb{E}$
(1)概念：一个随机变量的期望（或者期望值）指的是其概率分布集中趋势的一种度量。

(2)数学原理：
![](./images/数学期望的数学原理.png)

(3)期望的性质：
线性、非负性（如果随机变量非负）、独立性（如果随机变量是互相独立的）。注意后两个性质是有前提条件的。
![](./images/数学期望的性质.png)

(4)数学期望在RL中的应用：
主要用于评估plicy的性能。
![](./images/数学期望在RL中的应用.png)

### 1.1.2 优势函数 Advantage Function
（1）概念：
通过量化在特定状态下采取某个行动的相对质量，与在该状态下所有可能行动的平均预期质量之间的差距，来帮助提高策略优化过程中的稳定性和效率。

（2）数学原理
![](./images/优势函数的数学原理.png)

（3）优势函数的性质
相对比较、泛华性、信号可解释性。
![](./images/数学期望的性质.png)

（4）优势函数的RL应用
![](./images/优势函数的应用_1.png)
![](./images/优势函数的应用_2.png)
![](./images/优势函数的应用_3.png)

### 1.1.3 KL散度 Kullback-Leibler(KL) divergence
（1）概念：KL散度主要用于量化两个概率分布之间的差异。

（2）数学原理：
分为离散分布和连续分布两种情况。
![](./images/KL散度的数学原理.png)

特殊情况：
![](./images/KL散度的特殊计算情况.png)

（3）KL散度的性质
非负性、不对称性、与熵之间的关系、凸性。
![](./images/KL散度的性质.png)

（4）KL散度的应用例子
![](./images/KL散度的应用例子_1.png)

# 2.TRPO目标函数的求解过程
![](./images/TRPO求解的过程概览.jpg)

## 2.1 定义策略优化的目标：
策略优化的目标，就是最大化期望的期望函数的值。
![](./images/TRPO目标函数求解步骤_1.jpg)


## 2.2 定义代理目标：
![](./images/TRPO目标函数求解步骤_2.jpg)

这里之所以叫做代理目标，是因为TRPO并不按照1中的方式，直接优化目标函数，而是对一个新的代理目标进行优化，具体为什么优化代理目标就能得到优化原始目标同样的策略优化效果，具体的数学证明如下。
![](./images/TRPO代理目标函数与原目标函数的等价证明.png)



## 2.3 引入置信域约束：
引入置信域约束的主要原因是在代理目标函数的直接使用朴素梯度下降会导致策略的大幅度更新，导致泛化性差并且难以收敛。而TRPO使用KL散度解决了这个问题，限制了在一次更新中，策略更新的幅度，防止了灾难性的大幅度策略更新。
![](./images/TRPO目标函数求解步骤_3.jpg)



## 2.4 构造约束的优化问题：
![](./images/TRPO目标函数求解步骤_4.jpg)



## 2.5 使用自然梯度下降求解：
由于KL散度的约束导致无法直接求解，因此使用二次估计对KL散度进行近似。近似完成之后就可以使用自然梯度下降法来进行求解。
![](./images/TRPO目标函数求解步骤_5.jpg)

### 2.5.1 二次估计 quadratic approximation
在TRPO中的主要作用就是使用二阶泰勒展开对KL散度约束项进行估计。
![](./images/二次估计KL散度的数学过程.jpg)

### 2.5.2 Fisher information matrix(FIM)
FIM的主要作用是作为KL散度的Hessian(二阶导)近似，使得自然梯度能够高效计算。
![](.//images/FisherInformationMatrix.jpg)

### 2.5.3 Natural Gradient Descent
NGD在TRPO中的主要作用就是求解目标函数的。
![](./images/自然梯度下降法.jpg)

## 2.6 使用共轭梯度进行更新
使用共轭梯度算法原因主要是因为计算自然梯度的方向时，计算开销太大，这里的开销主要指的是计算矩阵的逆的时候，开销很大。
![](./images/TRPO目标函数求解步骤_6.jpg)

### 2.6.1 共轭梯度算法Cojugate gradient algorithm
![](./images/共轭梯度算法原理.jpg)

## 2.7 使用线搜索来进行最终的更新
使用线搜索的主要原因是为找到最佳的step size $\alpha$ <br>
![](./images/TRPO目标函数求解步骤_7.jpg)

### 2.7.1 Line Search
线搜索是一种优化方法，通常用于在给定方向下移动时找到最佳的step size。通常与基于梯度的算法结合，比如梯度下降，共轭梯度等。
![](./images/线搜索的数学原理.jpg)